In [57]:
# load style
from ROOT import gROOT

path = './../../'
# path = '/eos/user/y/youyou/'
gROOT.SetMacroPath(path+"atlasstyle/")
gROOT.LoadMacro("AtlasLabels.C")
gROOT.LoadMacro("AtlasStyle.C")
gROOT.LoadMacro("AtlasUtils.C")
from ROOT import SetAtlasStyle, ATLASLabel, myText, myMarkerText
SetAtlasStyle()


Applying ATLAS style settings...



In [58]:
# load lib
import ROOT
import os, sys
import numpy as np
from array import array
import pandas as pd

# import pandas as pd

In [59]:
# load files

# standard model
sm_file = path + 'EFT_MINI_TREES/SM.root'
sm_data = ROOT.TFile(sm_file,'READ')
sm_tree = sm_data.vbswy_tree

# O_INT & O_QUAD
eft_path = path + 'EFT_MINI_TREES/'
files = [(eft_path+i) for i in os.listdir(eft_path)]
files.sort()
files.remove(sm_file)
int_files = [i for i in files if 'INT' in i]
quad_files = [i for i in files if 'QUAD' in i]

# names of operators used
operators = [i.split('/')[-1].split('_')[0] for i in int_files]

In [67]:

# list of branches we interested in 
names = [g.GetName() for g in sm_tree.GetListOfBranches()]
truth_branch = [i for i in names if 'truth' in i]
truth_branch = list(dict.fromkeys(truth_branch))

is_reconst = lambda i: not(('truth' in i) or ('weight' in i) or ('pass_baseline' in i))
reconst_branch = [i for i in names if is_reconst(i)]
reconst_branch = list(dict.fromkeys(reconst_branch))

# randomly generate an arr of bins from 0 to 16440,
# with increasing bin size
bins = np.linspace(1.2,1.2,40).cumprod()
bins = np.insert(bins, 0, 0)

# generate max value dictionary for sm tree
max_sm_dict = {}
for branch in names:
    max_sm_dict[branch] = sm_tree.GetMaximum(branch)
    
max_sm_dict

{'weight': 0.3747520756248783,
 'jj_drap': 8.317871998076633,
 'jj_m': 7583.18091078032,
 'jj_pt': 2272.6573001713664,
 'jj_dphi_signed': 3.1413330361247063,
 'jj_truth_drap': 8.637828046803866,
 'jj_truth_m': 7121.392450608517,
 'jj_truth_pt': 2208.1405294690107,
 'jj_truth_dphi_signed': 3.141589105129242,
 'gam_pt': 1818.910625,
 'gam_eta': 2.3698120117187536,
 'gam_phi': 3.1414341926574707,
 'gam_e': 3555.0673493398303,
 'gam_truth_pt': 1848.5566727951737,
 'gam_truth_eta': 2.369648674150898,
 'gam_truth_phi': 3.141542155187217,
 'gam_truth_e': 3457.849,
 'lep_pt': 1340.063375,
 'lep_eta': 2.476752758026128,
 'lep_phi': 3.1415679454803467,
 'lep_e': 2493.646147428611,
 'lep_truth_pt': 1309.7969352030134,
 'lep_truth_eta': 2.469721069156157,
 'lep_truth_phi': 0.0,
 'lep_truth_e': 2395.708435971912,
 'j0_pt': 2669.738352457746,
 'j0_eta': 4.379240989684844,
 'j0_phi': 3.14153790473938,
 'j0_e': 5312.517503391469,
 'j0_truth_pt': 2709.8225,
 'j0_truth_eta': 4.4011039733886665,
 'j0_tru

In [61]:
cut_bins = np.linspace(1.05,1.05,100).cumprod()
cut_bins = np.insert(cut_bins, 0, 0)
cut_bins

array([  0.        ,   1.05      ,   1.1025    ,   1.157625  ,
         1.21550625,   1.27628156,   1.34009564,   1.40710042,
         1.47745544,   1.55132822,   1.62889463,   1.71033936,
         1.79585633,   1.88564914,   1.9799316 ,   2.07892818,
         2.18287459,   2.29201832,   2.40661923,   2.5269502 ,
         2.65329771,   2.78596259,   2.92526072,   3.07152376,
         3.22509994,   3.38635494,   3.55567269,   3.73345632,
         3.92012914,   4.1161356 ,   4.32194238,   4.53803949,
         4.76494147,   5.00318854,   5.25334797,   5.51601537,
         5.79181614,   6.08140694,   6.38547729,   6.70475115,
         7.03998871,   7.39198815,   7.76158756,   8.14966693,
         8.55715028,   8.98500779,   9.43425818,   9.90597109,
        10.40126965,  10.92133313,  11.46739979,  12.04076978,
        12.64280826,  13.27494868,  13.93869611,  14.63563092,
        15.36741246,  16.13578309,  16.94257224,  17.78970085,
        18.67918589,  19.61314519,  20.59380245,  21.62

In [68]:
def plot_2_branch(operator, branches, c=1, isTruth=True, cut_sm=True, isPDF=False):
    '''
    used default bins and assume all is log(no consider min)(x log didn't test islogy(lin50))
    '''
    
    # load files
    i = operators.index(operator)
    int_data = ROOT.TFile(int_files[i],'READ')
    int_tree = int_data.vbswy_tree
    quad_data = ROOT.TFile(quad_files[i], 'READ')
    quad_tree = quad_data.vbswy_tree

    def generate_output_path():
        folder = 'reconstructed/'
        if isTruth:
            folder = 'truth/'
        format = 'png'
        if isPDF:
            format = 'pdf'
        folder = './output_'+format+'/'+folder
        if not os.path.exists(folder):
            os.mkdir(folder)
        if cut_sm:
            return (folder+operator+'_cut_2D.'+format)
        return (folder+operator+'_2D.'+format)

    # ready to plot
    nvars = len(branches)   # number of variables to plot
    max_dict = {}
    for branch in branches:
        max_dict[branch] = max(int_tree.GetMaximum(branch), quad_tree.GetMaximum(branch), max_sm_dict[branch])

    can = ROOT.TCanvas('can1', '',4000,4000)
    can.Divide(nvars-1, nvars-1)

    # start plot
    for b in range(nvars):
        for a in range(b):
            ican = (b-1)*(nvars-1)+a+1  # use parts under diagonal of can
            can.cd(ican).SetRightMargin(0)
            can.cd(ican).SetLeftMargin(0)
            can.cd(ican).SetTopMargin(0)
            can.cd(ican).SetBottomMargin(0)
            if a == 0:
                can.cd(ican).SetLeftMargin(.15)
            elif a == b-1:
                can.cd(ican).SetRightMargin(.15)
            if b == nvars-1:
                can.cd(ican).SetBottomMargin(.15)      
            
            # variable name
            var_x = branches[ a ]
            var_y = branches[ b ]

            can.cd(ican).SetLogz(True)
            set_log_xy = lambda r: (r.split('_')[0] in ['lep', 'w', 'wy']) and not(isTruth)
            can.cd(ican).SetLogx(set_log_xy(var_x))
            can.cd(ican).SetLogy(set_log_xy(var_y))

            if cut_sm:
                divider_x = bins[-1] / (max_dict[var_x] - max_sm_dict[var_x])
                divider_y = bins[-1] / (max_dict[var_y] - max_sm_dict[var_y])
                bins_x = (bins/divider_x) + max_sm_dict[var_x]
                bins_y = (bins/divider_y) + max_sm_dict[var_y]
            else:
                divider_x = bins[-1] / max_dict[var_x]
                divider_y = bins[-1] / max_dict[var_y]
                bins_x = (bins/divider_x)
                bins_y = (bins/divider_y)

            h1 = ROOT.TH2F(var_x+var_y, ";"+var_x+";"+var_y, len(bins_x)-1, array('d',bins_x), len(bins_y)-1, array('d',bins_y))

            if isTruth:
                for i in range(sm_tree.GetEntries()):
                    sm_tree.GetEntry(i)
                    h1.Fill(getattr(sm_tree, var_x), getattr(sm_tree, var_y), sm_tree.weight)
                for i in range(int_tree.GetEntries()):
                    int_tree.GetEntry(i)
                    h1.Fill(getattr(int_tree, var_x), getattr(int_tree, var_y), int_tree.weight*c)
                for i in range(quad_tree.GetEntries()):
                    quad_tree.GetEntry(i)
                    h1.Fill(getattr(quad_tree, var_x), getattr(quad_tree, var_y), quad_tree.weight*c**2)
            else:
                for i in range(sm_tree.GetEntries()):
                    sm_tree.GetEntry(i)
                    tmp = int(sm_tree.pass_baseline_e) | int(sm_tree.pass_baseline_mu)
                    if tmp:
                        h1.Fill(getattr(sm_tree, var_x), getattr(sm_tree, var_y), sm_tree.weight)
                for i in range(int_tree.GetEntries()):
                    int_tree.GetEntry(i)
                    tmp = int(int_tree.pass_baseline_e) | int(int_tree.pass_baseline_mu)
                    if tmp:
                        h1.Fill(getattr(int_tree, var_x), getattr(int_tree, var_y), int_tree.weight*c)
                for i in range(quad_tree.GetEntries()):
                    quad_tree.GetEntry(i)
                    tmp = int(quad_tree.pass_baseline_e) | int(quad_tree.pass_baseline_mu)
                    if tmp:
                        h1.Fill(getattr(quad_tree, var_x), getattr(quad_tree, var_y), quad_tree.weight*c**2)
            h1.Draw()
            cr = h1.GetCorrelationFactor(1,2)
            #can.CreateText('correlation coefficient is %.3f \n'%cr)
            '''str = 'correlation coefficient is %.3f \n'%cr+var_x
            h1.GetXaxis().SetTitle(str)'''
            h1.DrawClone('Colz')
            t = ROOT.TLatex()
            t.SetNDC()
            t.SetTextColor( 1 )
            t.SetTextSize( 0.05 )
            t.SetTextAlign( 12 )
            t.DrawLatex( 0.2, 0.95, 'correlation coefficient is %.3f'%cr )
    can.cd()
    t = ROOT.TLatex()
    t.SetNDC()
    t.SetTextColor( 1 )
    t.SetTextSize( 0.08 )
    t.SetTextAlign( 12 )
    t.DrawLatex( 0.65, 0.75, operator )

    can.Update()
    can.Draw()
   
    can.Print('test.png')
    can.SaveAs(generate_output_path())

    #can.Clear()
    int_data.Close()
    quad_data.Close()

In [69]:
def plot_2_branch_cut_half(operator, branches, c=1, isTruth=True, isPDF=False):
    '''
    only y axis is cut
    '''
    
    # load files
    i = operators.index(operator)
    int_data = ROOT.TFile(int_files[i],'READ')
    int_tree = int_data.vbswy_tree
    quad_data = ROOT.TFile(quad_files[i], 'READ')
    quad_tree = quad_data.vbswy_tree

    def generate_output_path():
        folder = 'reconstructed/'
        if isTruth:
            folder = 'truth/'
        format = 'png'
        if isPDF:
            format = 'pdf'
        folder = './output_'+format+'/'+folder
        if not os.path.exists(folder):
            os.mkdir(folder)
        return (folder+operator+'_2D.'+format)

    # ready to plot
    nvars = len(branches)   # number of variables to plot
    max_dict = {}
    for branch in branches:
        max_dict[branch] = max(int_tree.GetMaximum(branch), quad_tree.GetMaximum(branch), max_sm_dict[branch])

    can = ROOT.TCanvas('can1', '',4000,4000)
    can.Divide(nvars-1, nvars-1)

    # start plot
    ican = 0
    for b in range(nvars):
        for a in range(nvars):
            if a==b:
                continue
            ican += 1
            can.cd(ican).SetRightMargin(0)
            can.cd(ican).SetLeftMargin(0)
            can.cd(ican).SetTopMargin(0)
            can.cd(ican).SetBottomMargin(0)
            if a == 0:
                can.cd(ican).SetLeftMargin(.15)
            elif a == nvars-1:
                can.cd(ican).SetRightMargin(.15)
            if b == nvars-1:
                can.cd(ican).SetBottomMargin(.15)      
            
            # variable name
            var_x = branches[ a ]
            var_y = branches[ b ]

            can.cd(ican).SetLogz(True)
            set_log_xy = lambda r: (r.split('_')[0] in ['lep', 'w', 'wy']) and not(isTruth)
            can.cd(ican).SetLogx(set_log_xy(var_x))
            can.cd(ican).SetLogy(set_log_xy(var_y))
        
            divider_y = bins[-1] / (max_dict[var_y] - max_sm_dict[var_y])
            bins_y = (bins/divider_y) + max_sm_dict[var_y]
            divider_x = bins[-1] / max_dict[var_x]
            bins_x = (bins/divider_x)

            h1 = ROOT.TH2F(var_x+var_y, ";"+var_x+'_cut'+";"+var_y, len(bins_x)-1, array('d',bins_x), len(bins_y)-1, array('d',bins_y))

            if isTruth:
                for i in range(sm_tree.GetEntries()):
                    sm_tree.GetEntry(i)
                    h1.Fill(getattr(sm_tree, var_x), getattr(sm_tree, var_y), sm_tree.weight)
                for i in range(int_tree.GetEntries()):
                    int_tree.GetEntry(i)
                    h1.Fill(getattr(int_tree, var_x), getattr(int_tree, var_y), int_tree.weight*c)
                for i in range(quad_tree.GetEntries()):
                    quad_tree.GetEntry(i)
                    h1.Fill(getattr(quad_tree, var_x), getattr(quad_tree, var_y), quad_tree.weight*c**2)
            else:
                for i in range(sm_tree.GetEntries()):
                    sm_tree.GetEntry(i)
                    tmp = int(sm_tree.pass_baseline_e) | int(sm_tree.pass_baseline_mu)
                    if tmp:
                        h1.Fill(getattr(sm_tree, var_x), getattr(sm_tree, var_y), sm_tree.weight)
                for i in range(int_tree.GetEntries()):
                    int_tree.GetEntry(i)
                    tmp = int(int_tree.pass_baseline_e) | int(int_tree.pass_baseline_mu)
                    if tmp:
                        h1.Fill(getattr(int_tree, var_x), getattr(int_tree, var_y), int_tree.weight*c)
                for i in range(quad_tree.GetEntries()):
                    quad_tree.GetEntry(i)
                    tmp = int(quad_tree.pass_baseline_e) | int(quad_tree.pass_baseline_mu)
                    if tmp:
                        h1.Fill(getattr(quad_tree, var_x), getattr(quad_tree, var_y), quad_tree.weight*c**2)
            h1.Draw()
            cr = h1.GetCorrelationFactor(1,2)
            h1.DrawClone('Colz')
            t = ROOT.TLatex()
            t.SetNDC()
            t.SetTextColor( 1 )
            t.SetTextSize( 0.05 )
            t.SetTextAlign( 12 )
            t.DrawLatex( 0.2, 0.95, 'correlation coefficient is %.3f'%cr )
    can.cd()
    t = ROOT.TLatex()
    t.SetNDC()
    t.SetTextColor( 1 )
    t.SetTextSize( 0.08 )
    t.SetTextAlign( 12 )
    t.DrawLatex( 0.65, 0.75, operator )

    can.Update()
    can.Draw()
   
    can.Print('test.png')
    can.SaveAs(generate_output_path())

In [73]:
def plot_2_branch_rank(operator, branches, c=1, isTruth=True, cut_sm=True, isPDF=True):
    '''
    ranked by correlation
    '''
    
    # load files
    i = operators.index(operator)
    int_data = ROOT.TFile(int_files[i],'READ')
    int_tree = int_data.vbswy_tree
    quad_data = ROOT.TFile(quad_files[i], 'READ')
    quad_tree = quad_data.vbswy_tree

    # dictionary of maximum of each variables
    max_dict = {}
    for branch in branches:
        max_dict[branch] = max(int_tree.GetMaximum(branch), quad_tree.GetMaximum(branch), max_sm_dict[branch])

    hs = []
    # add histogram
    for b in range(len(branches)):
        for a in range(b):    
            
            # variable name
            var_x = branches[ a ]
            var_y = branches[ b ]

            if cut_sm:
                divider_x = bins[-1] / (max_dict[var_x] - max_sm_dict[var_x])
                divider_y = bins[-1] / (max_dict[var_y] - max_sm_dict[var_y])
                bins_x = (bins/divider_x) + max_sm_dict[var_x]
                bins_y = (bins/divider_y) + max_sm_dict[var_y]
            else:
                divider_x = bins[-1] / max_dict[var_x]
                divider_y = bins[-1] / max_dict[var_y]
                bins_x = (bins/divider_x)
                bins_y = (bins/divider_y)

            h1 = ROOT.TH2F(var_x+var_y, ";"+var_x+";"+var_y, len(bins_x)-1, array('d',bins_x), len(bins_y)-1, array('d',bins_y))

            if isTruth:
                for i in range(sm_tree.GetEntries()):
                    sm_tree.GetEntry(i)
                    h1.Fill(getattr(sm_tree, var_x), getattr(sm_tree, var_y), sm_tree.weight)
                for i in range(int_tree.GetEntries()):
                    int_tree.GetEntry(i)
                    h1.Fill(getattr(int_tree, var_x), getattr(int_tree, var_y), int_tree.weight*c)
                for i in range(quad_tree.GetEntries()):
                    quad_tree.GetEntry(i)
                    h1.Fill(getattr(quad_tree, var_x), getattr(quad_tree, var_y), quad_tree.weight*c**2)
            else:
                for i in range(sm_tree.GetEntries()):
                    sm_tree.GetEntry(i)
                    tmp = int(sm_tree.pass_baseline_e) | int(sm_tree.pass_baseline_mu)
                    if tmp:
                        h1.Fill(getattr(sm_tree, var_x), getattr(sm_tree, var_y), sm_tree.weight)
                for i in range(int_tree.GetEntries()):
                    int_tree.GetEntry(i)
                    tmp = int(int_tree.pass_baseline_e) | int(int_tree.pass_baseline_mu)
                    if tmp:
                        h1.Fill(getattr(int_tree, var_x), getattr(int_tree, var_y), int_tree.weight*c)
                for i in range(quad_tree.GetEntries()):
                    quad_tree.GetEntry(i)
                    tmp = int(quad_tree.pass_baseline_e) | int(quad_tree.pass_baseline_mu)
                    if tmp:
                        h1.Fill(getattr(quad_tree, var_x), getattr(quad_tree, var_y), quad_tree.weight*c**2)
            h1.Draw()
            hs += [h1]

    hs = sorted(hs, key=lambda x: x.GetCorrelationFactor(1,2), reverse=True)

    #ready to plot
    nvars = len(branches)   # number of variables to plot
    can = ROOT.TCanvas('can1', '',4000,4000)
    nplots = int(np.ceil(((nvars-1) * nvars/2) **0.5))
    can.Divide(nplots, nplots)

    ican = 1
    for h in hs:
        can.cd(ican).SetRightMargin(0)
        can.cd(ican).SetLeftMargin(0.15)
        can.cd(ican).SetTopMargin(0)
        can.cd(ican).SetBottomMargin(0.15)
        if (ican % nplots == 0) or (ican == len(hs)):
            can.cd(ican).SetRightMargin(.15)
 
        can.cd(ican).SetLogz(True)
        can.cd(ican).SetLogx(1)
        can.cd(ican).SetLogy(1)
        h.DrawClone('Colz')
        t = ROOT.TLatex()
        t.SetNDC()
        t.SetTextColor( 1 )
        t.SetTextSize( 0.05 )
        t.SetTextAlign( 12 )
        t.DrawLatex( 0.2, 0.95, 'correlation coefficient is %.3f'%h.GetCorrelationFactor(1,2) )
        ican += 1

    can.Update()
    can.Draw()
    #can.Print('test.png')

    def generate_output_path():
        folder = 'reconstructed/'
        if isTruth:
            folder = 'truth/'
        format = 'png'
        if isPDF:
            format = 'pdf'
        folder = './output_'+format+'/'+folder
        if not os.path.exists(folder):
            os.mkdir(folder)
        if cut_sm:
            return (folder+operator+'_cut_2D_ranked.'+format)
        return (folder+operator+'_2D_ranked.'+format)
    can.SaveAs(generate_output_path())

    int_data.Close()
    quad_data.Close()

In [74]:
M_t = ['gam_truth_e', 'gam_truth_pt', 'j0_truth_pt', 'jj_truth_pt', 'lep_truth_pt', 'w_truth_mt','w_truth_pt','wy_truth_m', 'wy_truth_pt']
plot_2_branch_rank('M2', M_t, isTruth=True, isPDF=True)
M_t = ['gam_truth_e', 'gam_truth_pt', 'lep_truth_pt', 'w_truth_mt','w_truth_pt', 'wy_truth_m', 'wy_truth_pt']
plot_2_branch_rank('M5', M_t, isTruth=True, isPDF=True)
print('M truth completed')

T_t = ['gam_truth_e', 'gam_truth_pt', 'j0_truth_pt', 'jj_truth_pt', 'lep_truth_e', 'lep_truth_pt', 'w_truth_e', 'w_truth_mt', 'w_truth_pt', 'wy_truth_m', 'wy_truth_pt']
plot_2_branch_rank('T1', T_t, isTruth=True, isPDF=True)
plot_2_branch_rank('T6', T_t, isTruth=True, isPDF=True)
T_t = ['gam_truth_e', 'gam_truth_pt', 'lep_truth_e', 'lep_truth_pt', 'w_truth_e', 'w_truth_mt', 'w_truth_pt', 'wy_truth_m', 'wy_truth_pt']
plot_2_branch_rank('T5', T_t, isTruth=True, isPDF=True)
print('T truth completed')

M_r = ['gam_e', 'gam_pt', 'j0_pt', 'jj_pt', 'lep_e', 'lep_pt', 'w_e', 'w_mt', 'w_pt', 'wy_m', 'wy_pt']
plot_2_branch_rank('M2', M_r, isTruth=False, isPDF=True)
M_r = ['gam_e', 'gam_pt', 'j0_pt', 'jj_pt', 'lep_pt', 'w_mt', 'w_pt', 'wy_m', 'wy_pt']
plot_2_branch_rank('M5', M_r, isTruth=False, isPDF=True)
print('M recon completed')

T_r = ['gam_e', 'gam_pt', 'lep_e', 'lep_pt', 'w_e', 'w_mt', 'w_pt', 'wy_m', 'wy_pt']
plot_2_branch_rank('T1', T_r, isTruth=False, isPDF=True)
plot_2_branch_rank('T5', T_r, isTruth=False, isPDF=True)
plot_2_branch_rank('T6', T_r, isTruth=False, isPDF=True)
print('T recon completed')

M truth completed
T truth completed
M recon completed
T recon completed


Info in <TCanvas::Print>: pdf file ./output_pdf/truth/M2_cut_2D_ranked.pdf has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/truth/M5_cut_2D_ranked.pdf has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/truth/T1_cut_2D_ranked.pdf has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/truth/T6_cut_2D_ranked.pdf has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/truth/T5_cut_2D_ranked.pdf has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/reconstructed/M2_cut_2D_ranked.pdf has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/reconstructed/M5_cut_2D_ranked.pdf has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/reconstructed/T1_cut_2D_ranked.pdf has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/reconstructed/T5_cut_2D_ranked.pdf has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/reconstructed/T6_cut_2D_ranked.pdf has been created


In [71]:
M_t = ['gam_truth_e', 'gam_truth_pt', 'j0_truth_pt', 'jj_truth_pt', 'lep_truth_pt', 'w_truth_mt','w_truth_pt','wy_truth_m', 'wy_truth_pt']
plot_2_branch('M2', M_t, isTruth=True, isPDF=True)
M_t = ['gam_truth_e', 'gam_truth_pt', 'lep_truth_pt', 'w_truth_mt','w_truth_pt', 'wy_truth_m', 'wy_truth_pt']
plot_2_branch('M5', M_t, isTruth=True, isPDF=True)
print('M truth completed')

T_t = ['gam_truth_e', 'gam_truth_pt', 'j0_truth_pt', 'jj_truth_pt', 'lep_truth_e', 'lep_truth_pt', 'w_truth_e', 'w_truth_mt', 'w_truth_pt', 'wy_truth_m', 'wy_truth_pt']
plot_2_branch('T1', T_t, isTruth=True, isPDF=True)
plot_2_branch('T6', T_t, isTruth=True, isPDF=True)
T_t = ['gam_truth_e', 'gam_truth_pt', 'lep_truth_e', 'lep_truth_pt', 'w_truth_e', 'w_truth_mt', 'w_truth_pt', 'wy_truth_m', 'wy_truth_pt']
plot_2_branch('T5', T_t, isTruth=True, isPDF=True)
print('T truth completed')

M_r = ['gam_e', 'gam_pt', 'j0_pt', 'jj_pt', 'lep_e', 'lep_pt', 'w_e', 'w_mt', 'w_pt', 'wy_m', 'wy_pt']
plot_2_branch('M2', M_r, isTruth=False, isPDF=True)
M_r = ['gam_e', 'gam_pt', 'j0_pt', 'jj_pt', 'lep_pt', 'w_mt', 'w_pt', 'wy_m', 'wy_pt']
plot_2_branch('M5', M_r, isTruth=False, isPDF=True)
print('M recon completed')

T_r = ['gam_e', 'gam_pt', 'lep_e', 'lep_pt', 'w_e', 'w_mt', 'w_pt', 'wy_m', 'wy_pt']
plot_2_branch('T1', T_r, isTruth=False, isPDF=True)
plot_2_branch('T5', T_r, isTruth=False, isPDF=True)
plot_2_branch('T6', T_r, isTruth=False, isPDF=True)
print('T recon completed')

M truth completed
T truth completed
M recon completed
T recon completed


Warning in <TCanvas::Constructor>: Deleting canvas with same name: can1
Info in <TCanvas::Print>: png file test.png has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/truth/M2_cut_2D.pdf has been created
Info in <TCanvas::Print>: png file test.png has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/truth/M5_cut_2D.pdf has been created
Info in <TCanvas::Print>: png file test.png has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/truth/T1_cut_2D.pdf has been created
Info in <TCanvas::Print>: png file test.png has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/truth/T6_cut_2D.pdf has been created
Info in <TCanvas::Print>: png file test.png has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/truth/T5_cut_2D.pdf has been created
Info in <TCanvas::Print>: png file test.png has been created
Info in <TCanvas::Print>: pdf file ./output_pdf/reconstructed/M2_cut_2D.pdf has been created
Info in <TCanvas::Print>: png file test.png 